# PELIC Clean

<p>There are a few things to clean in the PELIC dataset:</p>
<ul>
    <li>The answers have different versions. We need to check whether or not the versions are different from each other, and select the versions we want to keep.</li>
    <li>There are columns with a large number of NA values. We need to decide how important they are before dropping them.</li>
    <li>The answers come from five different types of classes: Writing, Reading, Grammar, Listening, and Speaking. While it would seem intuitive that, for the purposes of this project, we would only want answers from the writing classes, there may be useful data from other types of classes.</li>
    <li>Several answers in the dataset consist of only one word or letter - these need to be filtered out.</li>
    <li>For the purposes of text classification, a minimum length of text needs to be decided upon.</li>
</ul>

In [1]:
# import libraries
import pandas as pd
import spacy

In [28]:
# spaCy will be used later on, so we're going to load this now
nlp = spacy.load('en_core_web_sm')

In [4]:
df = pd.read_csv('../data/PELIC_compiled.csv').drop(['Unnamed: 0'],axis=1)

In [5]:
df.head()

,answer_id,question_id,anon_id,course_id,version,created_date,text_len,answer,tokens,tok_lem_POS,...,semester_y,LCT_Form,LCT_Score,MTELP_Form,MTELP_I,MTELP_II,MTELP_III,MTELP_Conv_Score,Writing_Sample,question_type
0,1,5,eq0,149,1,2006-09-20 16:11:08,177,I met my friend Nife while I was studying in a...,"['I', 'met', 'my', 'friend', 'Nife', 'while', ...","[('I', 'I', 'PRP'), ('met', 'meet', 'VBD'), ('...",...,2006_spring,1.0,5.0,P,5.0,7.0,0.0,28.0,1.0,Paragraph writing
1,2,5,am8,149,1,2006-09-20 22:09:14,137,"Ten years ago, I met a women on the train betw...","['Ten', 'years', 'ago', ',', 'I', 'met', 'a', ...","[('Ten', 'ten', 'CD'), ('years', 'year', 'NNS'...",...,2006_spring,1.0,11.0,P,15.0,9.0,5.0,45.0,2.3,Paragraph writing
2,3,12,dk5,115,1,2006-09-21 10:16:17,63,In my country we usually don't use tea bags. F...,"['In', 'my', 'country', 'we', 'usually', 'do',...","[('In', 'in', 'IN'), ('my', 'my', 'PRP$'), ('c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paragraph writing
3,4,13,dk5,115,1,2006-09-21 10:16:17,6,I organized the instructions by time.,"['I', 'organized', 'the', 'instructions', 'by'...","[('I', 'I', 'PRP'), ('organized', 'organize', ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paragraph writing
4,5,12,ad1,115,1,2006-09-21 10:19:01,59,"First, prepare a port, loose tea, and cup.\nSe...","['First', ',', 'prepare', 'a', 'port', ',', 'l...","[('First', 'first', 'RB'), (',', ',', ','), ('...",...,2006_summer,1.0,23.0,R,22.0,30.0,14.0,81.0,2.0,Paragraph writing


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47667 entries, 0 to 47666
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   answer_id                   47667 non-null  int64  
 1   question_id                 47667 non-null  int64  
 2   anon_id                     47667 non-null  object 
 3   course_id                   47667 non-null  int64  
 4   version                     47667 non-null  int64  
 5   created_date                47667 non-null  object 
 6   text_len                    47667 non-null  int64  
 7   answer                      47664 non-null  object 
 8   tokens                      47667 non-null  object 
 9   tok_lem_POS                 47667 non-null  object 
 10  question_type_id            47538 non-null  float64
 11  question                    47241 non-null  object 
 12  allow_text                  47538 non-null  float64
 13  gender                      476

## Check for different versions of the same text

There are different versions of some texts, but we don't know what the difference is between the versions. We need to check for duplicate answer ids to find the multiple versions and check if they're different from each other.

In [7]:
df.version.value_counts()

version
1    42916
2     4154
3      597
Name: count, dtype: int64

In [8]:
# Find duplicate answer_id
duplicates = df[df.duplicated('answer_id', keep=False)]

# Display the duplicates
duplicates.head(6)

,answer_id,question_id,anon_id,course_id,version,created_date,text_len,answer,tokens,tok_lem_POS,...,semester_y,LCT_Form,LCT_Score,MTELP_Form,MTELP_I,MTELP_II,MTELP_III,MTELP_Conv_Score,Writing_Sample,question_type
24,25,10,gc5,114,1,2006-09-21 10:24:26,102,Last week I planned to go paintball match' but...,"['Last', 'week', 'I', 'planned', 'to', 'go', '...","[('Last', 'last', 'JJ'), ('week', 'week', 'NN'...",...,2006_fall,1.0,21.0,R,16.0,19.0,6.0,60.0,2.6,Paragraph writing
25,25,10,gc5,114,1,2006-09-21 10:24:26,102,Last week I planned to go paintball match' but...,"['Last', 'week', 'I', 'planned', 'to', 'go', '...","[('Last', 'last', 'JJ'), ('week', 'week', 'NN'...",...,2007_spring,1.0,24.0,R,15.0,16.0,7.0,57.0,3.0,Paragraph writing
454,492,44,gc5,102,1,2006-10-02 11:49:16,49,"screw, interval, concurrent, dence anticipate,...","['screw', ',', 'interval', ',', 'concurrent', ...","[('screw', 'screw', 'NN'), (',', ',', ','), ('...",...,2006_fall,1.0,21.0,R,16.0,19.0,6.0,60.0,2.6,Short answer
455,492,44,gc5,102,1,2006-10-02 11:49:16,49,"screw, interval, concurrent, dence anticipate,...","['screw', ',', 'interval', ',', 'concurrent', ...","[('screw', 'screw', 'NN'), (',', ',', ','), ('...",...,2007_spring,1.0,24.0,R,15.0,16.0,7.0,57.0,3.0,Short answer
1097,1140,107,gc5,114,1,2006-10-17 10:40:25,258,Limiting students on-line time may be seem a g...,"['Limiting', 'students', 'on-line', 'time', 'm...","[('Limiting', 'limit', 'VBG'), ('students', 's...",...,2006_fall,1.0,21.0,R,16.0,19.0,6.0,60.0,2.6,Paragraph writing
1098,1140,107,gc5,114,1,2006-10-17 10:40:25,258,Limiting students on-line time may be seem a g...,"['Limiting', 'students', 'on-line', 'time', 'm...","[('Limiting', 'limit', 'VBG'), ('students', 's...",...,2007_spring,1.0,24.0,R,15.0,16.0,7.0,57.0,3.0,Paragraph writing


In [10]:
# We can see above and below that there appears to be no difference between the two versions
print(df[df.answer_id == 25].answer[24])
print('\n')
print(df[df.answer_id == 25].answer[25])

Last week I planned to go paintball match' but I lived a healthy problem. I had hard lumbago last days. Therfore I asked my brother could you go instead of me? He accepted my order and he went paintball area yesterday, but yesterday was rainy and cold. On the other hand the other players who all of arabic spoked between native language. My brother didn't understand nothing what they spoked. Even if he played this game, he hadn't a good time. He came to home with funny statue. All wears was coloured and wet. He slept very well last night.


Last week I planned to go paintball match' but I lived a healthy problem. I had hard lumbago last days. Therfore I asked my brother could you go instead of me? He accepted my order and he went paintball area yesterday, but yesterday was rainy and cold. On the other hand the other players who all of arabic spoked between native language. My brother didn't understand nothing what they spoked. Even if he played this game, he hadn't a good time. He came 

To filter out the different versions, we're only going to keep one answer from each version by ensuring that all answer_id's in the dataframe are unique.

In [12]:
# Find unique answer values
unique_answers = df['answer'].value_counts() == 1

# Get the index of unique answer values
unique_index = unique_answers[unique_answers].index

# Filter the DataFrame to keep only rows with unique answer values
unique_df = df[df['answer'].isin(unique_index)]

unique_df.head()

,answer_id,question_id,anon_id,course_id,version,created_date,text_len,answer,tokens,tok_lem_POS,...,semester_y,LCT_Form,LCT_Score,MTELP_Form,MTELP_I,MTELP_II,MTELP_III,MTELP_Conv_Score,Writing_Sample,question_type
0,1,5,eq0,149,1,2006-09-20 16:11:08,177,I met my friend Nife while I was studying in a...,"['I', 'met', 'my', 'friend', 'Nife', 'while', ...","[('I', 'I', 'PRP'), ('met', 'meet', 'VBD'), ('...",...,2006_spring,1.0,5.0,P,5.0,7.0,0.0,28.0,1.0,Paragraph writing
1,2,5,am8,149,1,2006-09-20 22:09:14,137,"Ten years ago, I met a women on the train betw...","['Ten', 'years', 'ago', ',', 'I', 'met', 'a', ...","[('Ten', 'ten', 'CD'), ('years', 'year', 'NNS'...",...,2006_spring,1.0,11.0,P,15.0,9.0,5.0,45.0,2.3,Paragraph writing
2,3,12,dk5,115,1,2006-09-21 10:16:17,63,In my country we usually don't use tea bags. F...,"['In', 'my', 'country', 'we', 'usually', 'do',...","[('In', 'in', 'IN'), ('my', 'my', 'PRP$'), ('c...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paragraph writing
3,4,13,dk5,115,1,2006-09-21 10:16:17,6,I organized the instructions by time.,"['I', 'organized', 'the', 'instructions', 'by'...","[('I', 'I', 'PRP'), ('organized', 'organize', ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paragraph writing
4,5,12,ad1,115,1,2006-09-21 10:19:01,59,"First, prepare a port, loose tea, and cup.\nSe...","['First', ',', 'prepare', 'a', 'port', ',', 'l...","[('First', 'first', 'RB'), (',', ',', ','), ('...",...,2006_summer,1.0,23.0,R,22.0,30.0,14.0,81.0,2.0,Paragraph writing


In [14]:
# Double check to make sure that there are no more duplicate answer_ids
duplicates = unique_df[unique_df.duplicated('answer_id', keep=False)]

# Display the duplicates
duplicates.head()

,answer_id,question_id,anon_id,course_id,version,created_date,text_len,answer,tokens,tok_lem_POS,...,semester_y,LCT_Form,LCT_Score,MTELP_Form,MTELP_I,MTELP_II,MTELP_III,MTELP_Conv_Score,Writing_Sample,question_type


## Select answers based on class and answer types

Answers from every class type allow for open answers, so we can keep data from all class types. The short answer question type has open-text answers as well as non-open-text answers (e.g., multiple choice). So, the most logical thing to do here is simply to drop all answers that don't allow for open text.

In [17]:
# Firstly, let's look at the class (course) types
unique_df.class_id.value_counts()
# w = writing
# r = reading
# g = grammar
# l = listesing
# s = speaking

class_id
w    13098
r    12651
g    10267
l      889
s       37
Name: count, dtype: int64

In [18]:
# Look at the question types for each type of class
writing_question_types = unique_df.groupby(['class_id', 'question_type'])['allow_text'].value_counts().unstack(fill_value=0)
print(writing_question_types)

allow_text                    0.0   1.0
class_id question_type                 
g        Audio recording        0     1
         Essay                  0    55
         Fill-in-the-blank      0   423
         Paragraph writing      0  6410
         Sentence completion    0   423
         Short answer           0  2955
l        Essay                  0    39
         Paragraph writing      0   582
         Short answer          56   200
r        Essay                  0   104
         Fill-in-the-blank     11     0
         Paragraph writing      0  3423
         Short answer         233  8861
         Word bank              7     0
s        Short answer           0    37
w        Essay                  0  3136
         Paragraph writing      0  5190
         Short answer           0  4759


In [19]:
# Conserve open-text answers
unique_df = unique_df[unique_df.allow_text == 1]

In [20]:
unique_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36598 entries, 0 to 47662
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   answer_id                   36598 non-null  int64  
 1   question_id                 36598 non-null  int64  
 2   anon_id                     36598 non-null  object 
 3   course_id                   36598 non-null  int64  
 4   version                     36598 non-null  int64  
 5   created_date                36598 non-null  object 
 6   text_len                    36598 non-null  int64  
 7   answer                      36598 non-null  object 
 8   tokens                      36598 non-null  object 
 9   tok_lem_POS                 36598 non-null  object 
 10  question_type_id            36598 non-null  float64
 11  question                    36304 non-null  object 
 12  allow_text                  36598 non-null  float64
 13  gender                      36598 no

## Deal with Null Values

<p>Now, we need to drop null values from the dataset, but also conserve as much information that we're interested in as possible for both X and y values. Intuitively, it would seem that 'level' would be the best y; however, this is the level of the class that they student was in at the time of writing the answer - it doesn't necessarily indicate the level of the answer itself. Further, the level of the class doesn't line up neatly with a distinct CEFR level. We could be better off using one of the Michigan Test of English Language Proficiency (MTELP) scores, or the 'Writing Sample' column as our y.</p>
<br>
<table>
    <tr>
        <th>level_id</th>
        <th>Level description</th>
        <th>CEFR level</th>
    </tr>
    <tr>
        <td>2</td>
        <td>Pre-Intermediate</td>
        <td>A2/B1</td>
    </tr>
    <tr>
        <td>3</td>
        <td>Intermediate</td>
        <td>B1</td>
    </tr>
    <tr>
        <td>4</td>
        <td>Upper-Intermediate</td>
        <td>B1+/B2</td>
    </tr>
    <tr>
        <td>5</td>
        <td>Advanced</td>
        <td>B2+/C1</td>
        <td></td>
    </tr>
</table>
<br>
<table>
    <tr>
        <th>Column Name</th>
        <th>Description</th>
    </tr>
    <tr>
        <td>MTELP_I</td>
        <td>Grammar section</td>
    </tr>
    <tr>
        <td>MTELP_II</td>
        <td>Reading section</td>
    </tr>
    <tr>
        <td>MTELP_III</td>
        <td>Listening section</td>
    </tr>
    <tr>
        <td>MTELP_Conv_Score</td>
        <td>Total combined score</td>
    </tr>
    <tr>
        <td>Writing_Sample</td>
        <td>In-house writing test score (scale of 1-6)</td>
    </tr>
</table>
<br>
<p> However, for the purposes of this project, we're going to assume that students were placed in their level based on their test scores, and thus level must be a reasonably good y. This will also help us conserve data, since there are several null values in the test score columns.</p>
<p>We can see that the 'question' column has 36,304, which is 294 fewer than the 'answer' column; however, we may need the 'question' column later.</p>
<p>That being said, we're going to drop every column that has fewer than 36,304 non-null values.</p>

In [22]:
unique_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36598 entries, 0 to 47662
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   answer_id                   36598 non-null  int64  
 1   question_id                 36598 non-null  int64  
 2   anon_id                     36598 non-null  object 
 3   course_id                   36598 non-null  int64  
 4   version                     36598 non-null  int64  
 5   created_date                36598 non-null  object 
 6   text_len                    36598 non-null  int64  
 7   answer                      36598 non-null  object 
 8   tokens                      36598 non-null  object 
 9   tok_lem_POS                 36598 non-null  object 
 10  question_type_id            36598 non-null  float64
 11  question                    36304 non-null  object 
 12  allow_text                  36598 non-null  float64
 13  gender                      36598 no

In [24]:
# Select columns to keep
columns_to_keep = ['answer_id', 'question_id', 'anon_id', 'course_id',
       'created_date', 'text_len', 'answer', 'tokens', 'tok_lem_POS',
       'question_type_id', 'question', 'allow_text', 'gender',
       'L1', 'class_id', 'level', 'question_type']

# Put them in a new df
cleaned_df = unique_df[columns_to_keep].dropna()

# Drop na values
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36304 entries, 0 to 47662
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   answer_id         36304 non-null  int64  
 1   question_id       36304 non-null  int64  
 2   anon_id           36304 non-null  object 
 3   course_id         36304 non-null  int64  
 4   created_date      36304 non-null  object 
 5   text_len          36304 non-null  int64  
 6   answer            36304 non-null  object 
 7   tokens            36304 non-null  object 
 8   tok_lem_POS       36304 non-null  object 
 9   question_type_id  36304 non-null  float64
 10  question          36304 non-null  object 
 11  allow_text        36304 non-null  float64
 12  gender            36304 non-null  object 
 13  L1                36304 non-null  object 
 14  class_id          36304 non-null  object 
 15  level             36304 non-null  int64  
 16  question_type     36304 non-null  object 
dty

## Filter out short answers

<p>We've already removed answer types that don't allow for open text; however, we can see from the text_len column that the minimum text length (determined by the number of tokens) is still zero. It appears that these answers, despite being from open text fields, are mainly numbers.</p>
<p>It's possible that there are other types of very short answers in the dataset, so we're going to use spaCy to make sure that each answer contains at least one subject and one verb.</p>
<p>This reduces the number of rows from 36,304 to 31,099.</p>

In [25]:
cleaned_df.text_len.describe()

count    36304.000000
mean       107.280603
std        165.353823
min          0.000000
25%         12.000000
50%         37.000000
75%        135.000000
max       2432.000000
Name: text_len, dtype: float64

In [27]:
cleaned_df[cleaned_df.text_len == 0].head()

,answer_id,question_id,anon_id,course_id,created_date,text_len,answer,tokens,tok_lem_POS,question_type_id,question,allow_text,gender,L1,class_id,level,question_type
11348,11629,1656,ec9,219,2007-07-26 00:06:09,0,71.5%,"['71.5', '%']","[('71.5', '71.5', 'CD'), ('%', '%', 'NN')]",2.0,What will the payroll tax rate in Italy be in ...,1.0,Male,Korean,r,5,Short answer
11360,11641,1656,bi6,219,2007-07-26 00:22:26,0,71.5%,"['71.5', '%']","[('71.5', '71.5', 'CD'), ('%', '%', 'NN')]",2.0,What will the payroll tax rate in Italy be in ...,1.0,Male,Arabic,r,5,Short answer
11558,11845,1717,ac5,277,2007-09-23 19:19:32,0,40,['40'],"[('40', '40', 'CD')]",2.0,How many languages are spoken now in southeast...,1.0,Male,Korean,r,5,Short answer
33084,33480,4387,gz5,617,2010-03-01 19:35:21,0,1981-85.,"['1981', '-', '85', '.']","[('1981', '1981', 'CD'), ('-', '-', ':'), ('85...",2.0,Which year has the most accidental deaths?,1.0,Female,Chinese,r,4,Short answer
33185,33617,4387,ge5,617,2010-03-02 00:20:54,0,1981-1985,"['1981', '-', '1985']","[('1981', '1981', 'CD'), ('-', '-', ':'), ('19...",2.0,Which year has the most accidental deaths?,1.0,Male,Spanish,r,4,Short answer


In [29]:
# Make sure that the answer column is a string
cleaned_df['answer'] = cleaned_df['answer'].astype('string')

# Define a function to filter the data for answers that contain at least one subject and verb
def contains_subject_and_verb(text):
    '''
    Checks to see if a document contains
    at least one subject and one verb
    '''
    doc = nlp(text)
    # Check if the text contains at least one subject and one verb
    return any(token.dep_ == "nsubj" for token in doc) and any(token.pos_ == "VERB" for token in doc)

def filter_rows_with_subject_and_verb(df):
    '''
    Applies the contains_subject_and_verb function
    '''
    # Apply the contains_subject_and_verb function to each row in the 'answer' column
    mask = df['answer'].apply(contains_subject_and_verb)
    # Filter the DataFrame to keep only the rows where the condition is True
    return df[mask]

# Apply the function
cleaned_df = filter_rows_with_subject_and_verb(cleaned_df)

In [30]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31099 entries, 0 to 47662
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   answer_id         31099 non-null  int64  
 1   question_id       31099 non-null  int64  
 2   anon_id           31099 non-null  object 
 3   course_id         31099 non-null  int64  
 4   created_date      31099 non-null  object 
 5   text_len          31099 non-null  int64  
 6   answer            31099 non-null  string 
 7   tokens            31099 non-null  object 
 8   tok_lem_POS       31099 non-null  object 
 9   question_type_id  31099 non-null  float64
 10  question          31099 non-null  object 
 11  allow_text        31099 non-null  float64
 12  gender            31099 non-null  object 
 13  L1                31099 non-null  object 
 14  class_id          31099 non-null  object 
 15  level             31099 non-null  int64  
 16  question_type     31099 non-null  object 
dty

## Add a column to indicate the dataset

We're going to add a column to indicate which dataset it comes from so that we can identify it after it's been merged with other datasets.

In [32]:
cleaned_df['dataset'] = 'PELIC'

In [33]:
cleaned_df.to_csv('../data/PELIC_cleaned.csv')